In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize

from sklearn.naive_bayes import MultinomialNB

from sklearn import metrics
from sklearn.decomposition import LatentDirichletAllocation
from gensim import corpora

In [44]:
import scrape as sc
from datetime import datetime

In [45]:
#load the data
scraper = sc.Scrape()
comments = scraper.return_comments_date(datetime(2016, 6, 20))

In [46]:
#test out the comments
print(len(comments)) #https://news.ycombinator.com/front?day=2015-04-09
print(len(comments[0])) #https://news.ycombinator.com/item?id=11941130

30
102


In [47]:
#flatten the list
comments = sum(comments, [])
print(comments[:3])

In [53]:
#basic stop words
from stop_words import get_stop_words
stop_words = get_stop_words('en')

#flatten the comments list

txts = [[word for word in sentence.lower().split() if word not in stop_words]
        for sentence in comments]

#collapse into tokens
from collections import Counter
frequency = Counter([word for txt in txts for word in txt])
tokens = [[token for token in txt if frequency[token] > 1] for txt in txts]

[['honestly', 'happy', 'stop', 'windows', '10', 'turning', 'laptop', 'hair', 'randomly', 'otherwise', 'entirely', 'thing', 'sitting', 'nothing,', 'suddenly', 'fan', 'will', 'start', 'cpu', 'utilization'], ['apparently', 'windows', '10', 'likes', 'use', 'idle', 'time', 'test', 'desktop', 'causing', 'rev', 'fan', 'idle', 'minutes.'], ['seriously', 'earth', 'happen?', 'even', 'edge', 'ends', 'rest', 'os', 'going', 'ensure', 'will', 'get', '3', 'hours', 'battery', 'life', 'laptop.'], ['windows', 'update', 'via', 'problem', 'machines', 'support.', 'randomly', '100%', 'cpu', 'minutes'], ['use', 'specific', 'pc', 'around', 'week,', 'basically', 'unusable', 'startup.', '100%', 'around', '20', 'minutes,', 'every', 'now', 'spike', 'hell', 'microsoft', 'playing', 'at,', 'process', 'switching', 'computers', 'linux.'], ['well,', 'one', 'thing', 'can', 'sure', 'will', 'never', 'happen', 'service', 'bothers', 'you,', 'can', 'always', 'reschedule', 'turn', 'off.', 'good', 'like', 'updates', 'windows',

In [54]:
#make the dictionary mapping
dict_gensim = corpora.Dictionary(tokens)
print(dict_gensim)

Dictionary(9977 unique tokens: ['counter?', '10k', 'universe', 'gap', 'teleport']...)


In [59]:
from gensim import models
tfidf = models.TfidfModel([dict_gensim.doc2bow(txt) for txt in txts])